In [140]:
import pandas as pd
import altair as alt
from scipy import stats
import numpy as np

```python
        state.now = parseFloat(values[0]);
        state.rpm = parseFloat(values[1]);
        state.speed = parseFloat(values[2]);
        state.tacho = parseFloat(values[3]);
        state.distance = parseFloat(values[4]);
        state.voltage = parseFloat(values[5]);
        state.temp_mos = parseFloat(values[6]);
        state.temp_motor = parseFloat(values[7]);
        state.current_motor = parseFloat(values[8]);
        state.current_batt = parseFloat(values[9]);
        state.net_energy = parseFloat(values[23]);
        state.efficiency = parseFloat(values[24]);
        state.avg_speed = parseFloat(values[25]);
        state.elapsed = parseFloat(values[26]);
        state.time_remaining = parseFloat(values[27]);
        state.fault_code = parseFloat(values[20])
```

In [141]:
def urban_theme():
    markColor = "#1696d2"
    axisColor = "#000000"
    backgroundColor = "#FFFFFF"
    font = "Lato"
    labelFont = "Lato"
    sourceFont = "Lato"
    gridColor = "#DEDDDD"
    main_palette = ["#1696d2", 
                    "#d2d2d2",
                    "#000000", 
                    "#fdbf11", 
                    "#ec008b", 
                    "#55b748", 
                    "#5c5859", 
                    "#db2b27", 
                   ]
    sequential_palette = ["#cfe8f3", 
                          "#a2d4ec", 
                          "#73bfe2", 
                          "#46abdb", 
                          "#1696d2", 
                          "#12719e", 
                         ]
    return {
        "width": 685,
        "height": 380,   
#        "autosize": "fit",
        "config": {
            "title": {
                "anchor": "start",
                "fontSize": 18,
                "font": font,
                "fontColor": "#000000"
            },
            "axisX": {
               "domain": True,
               "domainColor": axisColor,
               "domainWidth": 1,
               "grid": False,
               "labelFontSize": 12,
               "labelFont": labelFont,
               "labelAngle": 0,
               "tickColor": axisColor,
               "tickSize": 5,
               "titleFontSize": 12,
               "titlePadding": 10,
               "titleFont": font,
               "title": "",
           },
           "axisY": {
               "domain": False,
               "grid": True,
               "gridColor": gridColor,
               "gridWidth": 1,
               "labelFontSize": 12,
               "labelFont": labelFont,
               "labelPadding": 8,
               "ticks": False,
               "titleFontSize": 12,
               "titlePadding": 10,
               "titleFont": font,
               "titleAngle": 0,
               "titleY": -10,
               "titleX": 18,
           },
           "background": backgroundColor,
           "legend": {
               "labelFontSize": 12,
               "labelFont": labelFont,
               "symbolSize": 100,
               "symbolType": "square",
               "titleFontSize": 12,
               "titlePadding": 10,
               "titleFont": font,
               "title": "",
               "orient": "top-left",
               "offset": 0,
           },
           "view": {
               "stroke": "transparent",
           },
           "range": {
               "category": main_palette,
               "diverging": sequential_palette,
           },
           "area": {
               "fill": markColor,
           },
           "line": {
               "color": markColor,
               "stroke": markColor,
               "strokewidth": 5,
           },
           "trail": {
               "color": markColor,
               "stroke": markColor,
               "strokeWidth": 0,
               "size": 1,
           },
           "path": {
               "stroke": markColor,
               "strokeWidth": 0.5,
           },
           "point": {
               "filled": True,
           },
           "text": {
               "font": sourceFont,
               "color": markColor,
               "fontSize": 11,
               "align": "right",
               "fontWeight": 400,
               "size": 11,
           }, 
           "bar": {
                "size": 40,
                "binSpacing": 1,
                "continuousBandSize": 30,
                "discreteBandSize": 30,
                "fill": markColor,
                "stroke": False,
            }, 
       },
    }
    
    
alt.themes.register("my_custom_theme", urban_theme)
alt.themes.enable("my_custom_theme")

ThemeRegistry.enable('my_custom_theme')

In [142]:
df = pd.read_csv("drive-download-20240226T231824Z-001/Run1.csv", header=None)
rename_dict = {
    0: "now",
    1: "rpm",
    2: "speed",
    3: "tacho",
    4: "distance",
    8: "current_motor",
    9: "current_bat"
}
df = df.rename(columns=rename_dict)

In [143]:
def mask_df(df):
    start = df["now"][0]
    df["time"] = df["now"] - start
    run_df = df[df["speed"] > 0]
    start_time = run_df["time"].iloc[0]
    end_time = run_df["time"].iloc[-1]
    masked = df[(df["time"] > start_time) & (df["time"] < end_time)]
    return masked

In [150]:
masked = mask_df(df)
#rolling average over speeds
def graph_attribute(attribute, masked):
    copy = masked.copy()
    copy = copy[copy[attribute] < 50]
    copy["rolling"] = copy[attribute].rolling(window=3).mean()
    copy = copy[copy["rolling"].notna()]
    copy = copy[copy[attribute] > 1]
    chart = alt.Chart(copy[["time", "rolling"]]).mark_line().encode(
        x=alt.X("time", title="Time (s)"),
        y=alt.Y("rolling", title="Velocity (m/s)"))
    display(chart)

masked = masked[(masked["time"] > 150) & (masked["time"] < 200)]
graph_attribute("current_motor", masked)

alt.Chart(...)